In [136]:
import numpy as np
import pandas as pd

In [137]:
train_data = pd.read_csv("data/train_data.csv",sep='\t',header=None)
songs = pd.read_csv("data/songs.csv",sep='\t',header=None)
users = pd.read_csv("data/users.csv",sep='\t',header=None)
test_data = pd.read_csv("data/test_data.csv",sep='\t',header=None)

In [138]:
train_data = train_data.rename(columns={train_data.columns[0]:"user",train_data.columns[1]:"song",train_data.columns[2]:"number"})
test_data = test_data.rename(columns={test_data.columns[0]:"user",test_data.columns[1]:"song",test_data.columns[2]:"number"})
songs = songs.rename(columns={songs.columns[0]:"song"})
users = users.rename(columns={users.columns[0]:"user"})


In [139]:
users["user_id"] = ["user_" +str(i) for i in range(5000)]
songs["song_id"] = ["song_" +str(i) for i in range(49564)]

In [140]:
def transformation(data, users, songs):
    tmp =pd.merge(data,users,how='left',left_on=data.user, right_on=users.user)
    tmp1 = pd.merge(tmp,songs,how='left',left_on=tmp.song, right_on=songs.song)
    return tmp1[tmp1.columns[[4,6,2]]]

In [141]:
train_data = transformation(train_data,users,songs)
test_data = transformation(test_data, users, songs)

In [142]:
n,m = len(users),len(songs)

In [143]:
matrix = np.zeros((n,m))


In [144]:
train_data['rate'] = 1
train_data.rate[(train_data.number<=2)&(train_data.number>=1)] = 2

/Users/yuxiangwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [145]:
train_data.rate[(train_data.number>2)&(train_data.number<=3)]=3
train_data.rate[(train_data.number>3)&(train_data.number<=5)]=4
train_data.rate[(train_data.number>5)&(train_data.number<=8)]=5
train_data.rate[(train_data.number>8)&(train_data.number<=12)]=6
train_data.rate[(train_data.number>12)&(train_data.number<=18)]=7
train_data.rate[(train_data.number>18)]=8
train_data

/Users/yuxiangwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/yuxiangwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/yuxiangwang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/yuxiang

,user_id,song_id,number,rate
0,user_3873,song_33722,1,2
1,user_3873,song_30052,1,2
2,user_3873,song_40941,5,4
3,user_3873,song_27675,1,2
4,user_3873,song_14539,3,3
5,user_3873,song_25141,1,2
6,user_3873,song_6887,19,8
7,user_3873,song_14598,2,2
8,user_3873,song_30457,1,2
9,user_3873,song_12558,6,5


In [146]:
def to_matrix(data,matrix):
    for r in train_data.iterrows():
        tmp = r[1]
        row = int(tmp[0].split("_")[-1])
        col = int(tmp[1].split("_")[-1])
        rate = tmp[3]
        matrix[row][col] = rate
    return matrix

In [147]:
V_train = to_matrix(train_data,matrix)

In [148]:
V_test = to_matrix(test_data, matrix)

In [165]:
def indicate(matrix):
    index_row,index_col = np.nonzero(matrix)
    M = np.zeros(matrix.shape)
    for i in range(len(index_row)):
        M[index_row[i]][index_col[i]] = 1
    return M
def normalization(matrix, max_rating ):
    matrix +=1
    matrix[matrix>max_rating+1] = max_rating
    matrix[matrix<1] = 1
    matrix[matrix<0] = 1
    return matrix
def SVD_incremental(V_train,feature_number,iteration_number,regu = 0.001,max_rating =7, learning_rate = 0.005):
    n,m = V_train.shape
    U = np.ones((n, feature_number))*0.01
    M = np.ones((m, feature_number))*0.01
    I = indicate(V_train)
    for j in range(iteration_number):
        MUL = np.dot(U,M.T)
        MUL = normalization(MUL, max_rating)
        tmp = np.multiply((V_train - MUL),I)
        delta_U = []
        delta_M = []
        for i in range(n):
            delta_U.append(np.sum(tmp[i]*M.T,axis=1))
        delta_U= np.array(delta_U) -regu*U

        for k in range(m):
            delta_M.append(np.sum(tmp.T[k]*U.T,axis=1))
        delta_M= np.array(delta_M) -regu*M
        if j>200:
            learning_rate = 0.004
        if j>400:
            learning_rate = 0.003
        if j>600:
            learning_rate = 0.002
        if j>800:
            learning_rate = 0.001
        if j>1000:
            learning_rate = 0.0005
        if j>1500:
            learning_rate = 0.0001
        
        U = U + delta_U*learning_rate
        M = M + delta_M *learning_rate
        print error(V_train,U,M,I,regu),j
    return U,M

In [166]:
def error(V_train, U,M,I,regu):
    MUL = V_train-np.dot(U,M.T)
    error = np.sum(I*(MUL**2))
    error = error 
    error = error * 0.5
    return error 

In [167]:
v= np.array([[1,3,2,0],[3,1,5,0],[4,5,3,2],[0,2,3,0],[0,0,4,0]])
v.shape

(5, 4)

In [168]:
SVD_incremental(v,feature_number=3,iteration_number=2000)

65.93253041
65.6091859221
63.7493991852
53.8743927945
21.2961778399
8.42015148759
37.104424129
7.97690222847
54.5011659863
17.1081481625
8.86765911212
27.7801871839
7.42122874705
56.9789324857
23.7994098985
7.32709454967
49.9491416196
13.1743013171
14.283294776
11.6764709755
22.5447516687
10.5558276571
56.5772125154
17.7947644749
10.4308261664
23.0750692668
7.86436023162
43.8320406571
9.18022226633
28.9614845912
9.11265455219
65.2640442774
40.9631340012
8.66164996103
40.5887509604
8.28295828827
47.7031401471
10.3640622387
26.3840591908
9.25352836577
63.2482683307
33.163769768
8.05929449397
55.5611306485
20.6510529086
7.54865744763
42.1111224333
8.40952423821
38.6268248063
8.62378293925
55.5687183664
17.6413390156
8.76664959105
29.0056947179
7.47882319108
57.559513747
24.8317806259
7.35350748653
52.1038176383
15.7044718912
10.2153267476
20.5820329319
7.72923566807
39.8533458346
7.96468747838
43.1759370766
8.79663689508
42.842062584
9.02677625136
44.8622043894
9.46999659495
38.1313555501

(array([[ 0.19356196,  0.19356196,  0.19356196],
        [ 1.10966768,  1.10966768,  1.10966768],
        [ 0.60879948,  0.60879948,  0.60879948],
        [ 0.52206515,  0.52206515,  0.52206515],
        [ 0.73476582,  0.73476582,  0.73476582]]),
 array([[ 0.76687409,  0.76687409,  0.76687409],
        [ 1.10000547,  1.10000547,  1.10000547],
        [ 0.53418081,  0.53418081,  0.53418081],
        [ 0.2629755 ,  0.2629755 ,  0.2629755 ]]))

In [43]:
SVD_incremental(V_train,feature_number=3,iteration_number=10)

KeyboardInterrupt: 